# Chapter 9: Database Systems and SQL for Data Analysis

This chapter introduces **relational databases** and **SQL** from a data analyst's perspective.
We'll use **SQLite** and Python so everything runs locally (no server setup).

## Learning Goals
By the end of this chapter, you will be able to:
- Understand tables, rows/columns, primary keys, and foreign keys
- Design a simple schema and understand normalization
- Write core SQL: `SELECT`, `WHERE`, `ORDER BY`
- Summarize data with `COUNT`, `SUM`, `AVG` and `GROUP BY`
- Combine tables with joins (INNER and LEFT) and answer questions with subqueries
- Use views and indexes for reusable queries and performance
- Connect Python to databases and execute SQL safely

## Prerequisites
- Basic Python knowledge (Chapter 2)
- Familiarity with pandas DataFrames (Chapter 4)

## Chapter Outline
1. Introduction to relational databases
2. Create a practice database (SQLite)
3. Database schema and normalization
4. SQL basics (SELECT, WHERE, ORDER BY)
5. Aggregations and GROUP BY
6. Joins and subqueries
7. Views (reusable queries)
8. Indexes and query performance
9. Connecting Python to databases
10. Mini-project: Sales report

## 9.1 Introduction to Relational Databases

A **relational database** stores data in **tables** (also called relations). Each table has:
- **Rows** (records): individual entries
- **Columns** (fields): attributes of each entry

Think of a table like an Excel spreadsheet, but with strict rules about structure.

### Why Use Databases Instead of CSV Files?

| Feature | CSV Files | Databases |
|---------|-----------|-----------|
| **Speed** | Slow for large data | Fast queries with indexes |
| **Relationships** | Manual linking | Built-in joins |
| **Data integrity** | No enforcement | Constraints & foreign keys |
| **Concurrent access** | Limited | Multiple users safely |
| **Updates** | Replace entire file | Update specific rows |

### When to Use What?
- **CSV files**: Quick analysis, small datasets, data sharing
- **Databases**: Large datasets, multiple related tables, applications, data integrity matters

### Database Management Systems (DBMS)
Popular relational databases include:
- **SQLite**: Lightweight, file-based, great for learning (we'll use this!)
- **PostgreSQL**: Powerful open-source, production-ready
- **MySQL**: Popular for web applications
- **SQL Server**: Microsoft's enterprise solution

> **💡 Tip:** SQLite comes built into Python—no installation needed!

In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display

pd.set_option('display.max_columns', 50)

## 9.2 Create a Practice Database (SQLite)

We'll build a tiny retail database with four tables:
- `customers` – who buys from us
- `products` – what we sell
- `orders` – when customers placed orders
- `order_items` – what products are in each order (the detail)

**Why multiple tables?** This mirrors real systems and lets us practice **joins** and **aggregations**.

### SQLite Data Types

SQLite uses these main data types:

| Type | Description | Example |
|------|-------------|---------|
| `INTEGER` | Whole numbers | `42`, `-10` |
| `REAL` | Decimal numbers | `3.14`, `99.99` |
| `TEXT` | Strings | `'Hello'`, `'2025-03-15'` |
| `BLOB` | Binary data | Images, files |
| `NULL` | Missing value | (empty) |

> **💡 Tip:** SQLite is flexible with types, but always declare them for clarity.

In [ ]:
# Create an in-memory SQLite database (self-contained, disappears when Python closes)
# If you want a persistent file database instead: sqlite3.connect('chapter09_demo.db')
conn = sqlite3.connect(':memory:')

# Important: enforce foreign key constraints in SQLite (disabled by default!)
conn.execute('PRAGMA foreign_keys = ON;')

print('Database connection created successfully!')
print(f'Connection object: {conn}')

## 9.3 Database Schema and Normalization

A **schema** is the database blueprint: table names, columns, data types, and constraints.

### Key Concepts

| Term | Definition | Example |
|------|------------|---------|
| **Primary Key (PK)** | Uniquely identifies each row | `customer_id` |
| **Foreign Key (FK)** | Links to another table's PK | `orders.customer_id → customers.customer_id` |
| **NOT NULL** | Column cannot be empty | `name TEXT NOT NULL` |
| **CHECK** | Validates data | `CHECK (price >= 0)` |

### Visual: Our Database Schema

```
┌─────────────┐       ┌─────────────┐
│  customers  │       │  products   │
├─────────────┤       ├─────────────┤
│ customer_id │ PK    │ product_id  │ PK
│ name        │       │ product_name│
│ city        │       │ category    │
│ signup_date │       │ unit_price  │
└──────┬──────┘       └──────┬──────┘
       │                     │
       │ 1:N                 │ 1:N
       ▼                     ▼
┌─────────────┐       ┌─────────────┐
│   orders    │       │ order_items │
├─────────────┤       ├─────────────┤
│ order_id    │ PK ◄──│ order_id    │ FK
│ customer_id │ FK    │ product_id  │ FK
│ order_date  │       │ quantity    │
│ status      │       │order_item_id│ PK
└─────────────┘       └─────────────┘
```

### Normalization (Beginner Version)

**Normalization** means storing each fact **once** in the right place to avoid redundancy.

**Bad design (not normalized):**
| order_id | customer_name | customer_city | product_name | quantity |
|----------|---------------|---------------|--------------|----------|
| 101 | Asha | Pune | Notebook | 2 |
| 102 | Asha | Pune | Pen Set | 1 |

*Problem: "Asha" and "Pune" are repeated. If Asha moves, we must update many rows!*

**Good design (normalized):**
- Customer info stored once in `customers` table
- Orders reference `customer_id` only

> **💡 Tip for Analysts:** You may not design schemas, but understanding them helps you write correct joins and trust your results.

In [ ]:
schema_sql = '''
CREATE TABLE customers (
  customer_id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  city TEXT NOT NULL,
  signup_date TEXT NOT NULL  -- store dates as ISO text: YYYY-MM-DD
);

CREATE TABLE products (
  product_id INTEGER PRIMARY KEY,
  product_name TEXT NOT NULL,
  category TEXT NOT NULL,
  unit_price REAL NOT NULL CHECK (unit_price >= 0)
);

CREATE TABLE orders (
  order_id INTEGER PRIMARY KEY,
  customer_id INTEGER NOT NULL,
  order_date TEXT NOT NULL,
  status TEXT NOT NULL,
  FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

CREATE TABLE order_items (
  order_item_id INTEGER PRIMARY KEY,
  order_id INTEGER NOT NULL,
  product_id INTEGER NOT NULL,
  quantity INTEGER NOT NULL CHECK (quantity > 0),
  FOREIGN KEY (order_id) REFERENCES orders(order_id),
  FOREIGN KEY (product_id) REFERENCES products(product_id)
);
'''

conn.executescript(schema_sql)
print('Tables created.')

## 9.4 SQL Basics (SELECT, WHERE, ORDER BY)

SQL (Structured Query Language) is the standard language for talking to databases.

### Basic Query Structure

```sql
SELECT column1, column2, ...   -- What columns to show
FROM table_name                -- Which table to query
WHERE condition                -- Filter rows (optional)
ORDER BY column ASC/DESC;      -- Sort results (optional)
```

### SQL Execution Order (Important!)

Even though we write `SELECT` first, SQL processes queries in this order:

1. `FROM` – Choose the table
2. `WHERE` – Filter rows
3. `GROUP BY` – Group rows (for aggregations)
4. `HAVING` – Filter groups
5. `SELECT` – Choose columns
6. `ORDER BY` – Sort results
7. `LIMIT` – Limit rows returned

> **⚠️ Common Mistake:** Trying to use column aliases from `SELECT` in `WHERE`. It won't work because `WHERE` runs before `SELECT`!

## Insert sample data

We’ll add a small dataset so queries return interesting results.

Tip: We’ll use parameterized inserts, which is safer and avoids quoting bugs.

In [ ]:
customers = [
    (1, 'Asha', 'Pune', '2025-01-10'),
    (2, 'Bilal', 'Delhi', '2025-02-05'),
    (3, 'Chen', 'Bengaluru', '2025-02-20'),
    (4, 'Dina', 'Mumbai', '2025-03-01'),
]

products = [
    (1, 'Notebook', 'Stationery', 60.0),
    (2, 'Pen Set', 'Stationery', 120.0),
    (3, 'USB Drive 64GB', 'Electronics', 550.0),
    (4, 'Coffee Mug', 'Home', 250.0),
    (5, 'Mouse', 'Electronics', 799.0),
]

orders = [
    (101, 1, '2025-03-12', 'delivered'),
    (102, 2, '2025-03-15', 'delivered'),
    (103, 1, '2025-03-20', 'cancelled'),
    (104, 3, '2025-03-22', 'delivered'),
    (105, 4, '2025-04-01', 'processing'),
]

order_items = [
    (1001, 101, 1, 2),
    (1002, 101, 2, 1),
    (1003, 102, 3, 1),
    (1004, 102, 1, 1),
    (1005, 103, 4, 2),
    (1006, 104, 5, 1),
    (1007, 104, 4, 1),
    (1008, 105, 2, 3),
]

conn.executemany('INSERT INTO customers VALUES (?, ?, ?, ?);', customers)
conn.executemany('INSERT INTO products VALUES (?, ?, ?, ?);', products)
conn.executemany('INSERT INTO orders VALUES (?, ?, ?, ?);', orders)
conn.executemany('INSERT INTO order_items VALUES (?, ?, ?, ?);', order_items)

conn.commit()
print('Sample data inserted.')

## Helper: run SQL and show results as a table

When learning SQL, you’ll run lots of queries.
This helper function returns a pandas DataFrame so results display nicely.

Beginner tip: we’ll also use **parameters** (`?`) in queries to avoid quoting mistakes and to keep code safe.

In [ ]:
def query_df(sql: str, params=None) -> pd.DataFrame:
    if params is None:
        params = ()
    return pd.read_sql_query(sql, conn, params=params)

# Quick sanity checks
display(query_df('SELECT * FROM customers ORDER BY customer_id;'))
display(query_df('SELECT * FROM products ORDER BY product_id;'))

In [ ]:
# Example 1: SELECT all columns (use sparingly in production!)
print("All customers:")
display(query_df('SELECT * FROM customers ORDER BY customer_id;'))

# Example 2: SELECT specific columns + ORDER BY
print("\nCustomers sorted by signup date:")
q = '''
SELECT name, city, signup_date
FROM customers
ORDER BY signup_date;
'''
display(query_df(q))

# Example 3: WHERE filter (parameterized for safety)
print("\nCustomers from Pune:")
q = '''
SELECT customer_id, name, city
FROM customers
WHERE city = ?
ORDER BY name;
'''
display(query_df(q, params=('Pune',)))

### Common SQL Operators for WHERE

| Operator | Description | Example |
|----------|-------------|---------|
| `=` | Equal | `WHERE city = 'Pune'` |
| `<>` or `!=` | Not equal | `WHERE status <> 'cancelled'` |
| `>`, `<`, `>=`, `<=` | Comparisons | `WHERE price >= 100` |
| `BETWEEN` | Range (inclusive) | `WHERE price BETWEEN 50 AND 200` |
| `IN` | List of values | `WHERE city IN ('Pune', 'Delhi')` |
| `LIKE` | Pattern matching | `WHERE name LIKE 'A%'` (starts with A) |
| `IS NULL` | Check for NULL | `WHERE email IS NULL` |
| `AND`, `OR`, `NOT` | Logical operators | `WHERE city = 'Pune' AND status = 'active'` |

In [ ]:
# More WHERE examples

# Using comparison operators
print("Products priced above 200:")
display(query_df('SELECT product_name, unit_price FROM products WHERE unit_price > 200;'))

# Using BETWEEN
print("\nProducts priced between 100 and 600:")
display(query_df('SELECT product_name, unit_price FROM products WHERE unit_price BETWEEN 100 AND 600;'))

# Using IN
print("\nOrders that are delivered or processing:")
display(query_df("SELECT order_id, status FROM orders WHERE status IN ('delivered', 'processing');"))

# Using LIKE (pattern matching)
print("\nProducts starting with 'P':")
display(query_df("SELECT product_name FROM products WHERE product_name LIKE 'P%';"))

### ✏️ Exercise 1: Basic SELECT Queries

Try these yourself before looking at the solutions:

1. List all products sorted by price (highest first)
2. Show only delivered orders, sorted by date (oldest first)
3. Find customers who signed up in February 2025
4. List products in the 'Electronics' category

In [ ]:
# Exercise 1 Solutions

# 1) Products sorted by price (highest first)
print("1. Products by price (descending):")
display(query_df('''
SELECT product_name, unit_price
FROM products
ORDER BY unit_price DESC;
'''))

# 2) Delivered orders only (oldest first)
print("\n2. Delivered orders by date:")
display(query_df('''
SELECT order_id, customer_id, order_date, status
FROM orders
WHERE status = 'delivered'
ORDER BY order_date;
'''))

# 3) Customers who signed up in February 2025
print("\n3. February 2025 signups:")
display(query_df('''
SELECT name, city, signup_date
FROM customers
WHERE signup_date LIKE '2025-02%';
'''))

# 4) Electronics products
print("\n4. Electronics category:")
display(query_df('''
SELECT product_name, unit_price
FROM products
WHERE category = 'Electronics';
'''))

## 9.5 Aggregations and GROUP BY

**Aggregation functions** summarize many rows into a single value.

### Common Aggregation Functions

| Function | Description | Example |
|----------|-------------|---------|
| `COUNT(*)` | Number of rows | `COUNT(*)` → 5 |
| `COUNT(column)` | Non-NULL values | `COUNT(email)` → 3 |
| `SUM(column)` | Total | `SUM(quantity)` → 10 |
| `AVG(column)` | Average | `AVG(price)` → 125.50 |
| `MIN(column)` | Minimum value | `MIN(price)` → 60 |
| `MAX(column)` | Maximum value | `MAX(price)` → 799 |

### GROUP BY

`GROUP BY` tells SQL how to group rows before calculating aggregates.

```sql
SELECT category, COUNT(*) AS product_count
FROM products
GROUP BY category;
```

### WHERE vs HAVING

This is a **very common source of confusion** for beginners!

| Clause | When it runs | Use for |
|--------|--------------|---------|
| `WHERE` | **Before** grouping | Filtering individual rows |
| `HAVING` | **After** grouping | Filtering aggregated results |

> **⚠️ Common Mistake:** Using `WHERE COUNT(*) > 2`. This will error! Use `HAVING COUNT(*) > 2` instead.

In [ ]:
# Basic aggregations (without GROUP BY - summarizes entire table)
print("Overall statistics:")
display(query_df('''
SELECT 
    COUNT(*) AS total_products,
    AVG(unit_price) AS avg_price,
    MIN(unit_price) AS min_price,
    MAX(unit_price) AS max_price
FROM products;
'''))

# COUNT with GROUP BY
print("\nOrders by status:")
display(query_df('''
SELECT status, COUNT(*) AS order_count
FROM orders
GROUP BY status
ORDER BY order_count DESC;
'''))

# SUM with GROUP BY
print("\nTotal quantity ordered per product:")
display(query_df('''
SELECT 
    p.product_name, 
    SUM(oi.quantity) AS total_ordered
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.product_name
ORDER BY total_ordered DESC;
'''))

In [ ]:
# WHERE vs HAVING demonstration

# WHERE: filter rows BEFORE grouping (e.g., only delivered orders)
print("Order count by status (only considering orders from March):")
display(query_df('''
SELECT status, COUNT(*) AS order_count
FROM orders
WHERE order_date LIKE '2025-03%'
GROUP BY status;
'''))

# HAVING: filter groups AFTER aggregation
print("\nCategories with 2 or more products:")
display(query_df('''
SELECT category, COUNT(*) AS product_count
FROM products
GROUP BY category
HAVING COUNT(*) >= 2
ORDER BY product_count DESC;
'''))

# Combining WHERE and HAVING
print("\nStatus with 2+ orders (from March only):")
display(query_df('''
SELECT status, COUNT(*) AS order_count
FROM orders
WHERE order_date LIKE '2025-03%'
GROUP BY status
HAVING COUNT(*) >= 2;
'''))

## 9.6 Joins and Subqueries

A **join** combines rows from multiple tables using matching keys. We use joins because normalized databases store data across tables.

### Types of Joins

| Join Type | Returns | Use When |
|-----------|---------|----------|
| `INNER JOIN` | Only matching rows from both tables | You want complete matches only |
| `LEFT JOIN` | All rows from left table + matches from right | Finding "orphan" or missing data |
| `RIGHT JOIN` | All rows from right table + matches from left | Less common, same as LEFT but reversed |
| `FULL OUTER JOIN` | All rows from both tables | Comparing two datasets |

### Visual: INNER vs LEFT JOIN

```
Table A         Table B          INNER JOIN        LEFT JOIN
┌───┬───┐      ┌───┬───┐        ┌───┬───┬───┐     ┌───┬───┬───┐
│ id│ x │      │ id│ y │        │ id│ x │ y │     │ id│ x │ y │
├───┼───┤      ├───┼───┤        ├───┼───┼───┤     ├───┼───┼───┤
│ 1 │ a │      │ 1 │ m │   →    │ 1 │ a │ m │     │ 1 │ a │ m │
│ 2 │ b │      │ 3 │ n │        │ 3 │ c │ n │     │ 2 │ b │NULL│
│ 3 │ c │      │ 4 │ o │        └───┴───┴───┘     │ 3 │ c │ n │
└───┴───┘      └───┴───┘                          └───┴───┴───┘
                                 (only matches)    (all from A)
```

> **💡 Tip:** Start with `INNER JOIN` for most analytics. Use `LEFT JOIN` when you need to find missing matches (e.g., customers who never ordered).

In [ ]:
# INNER JOIN: Orders with customer details
print("Orders with customer information (INNER JOIN):")
display(query_df('''
SELECT
  o.order_id,
  o.order_date,
  o.status,
  c.name AS customer_name,
  c.city
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
ORDER BY o.order_date;
'''))

# Multi-table join: orders -> order_items -> products
print("\nLine items with order and product details:")
display(query_df('''
SELECT
  o.order_id,
  o.order_date,
  c.name AS customer_name,
  p.product_name,
  p.category,
  oi.quantity,
  p.unit_price,
  (oi.quantity * p.unit_price) AS line_total
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
ORDER BY o.order_id, p.product_name;
'''))

### LEFT JOIN: Finding Missing Data

LEFT JOIN is especially useful for finding records that DON'T have matches - like products that have never been ordered.

In [ ]:
# LEFT JOIN: Show ALL products, even those never ordered
print("All products with their order counts (LEFT JOIN):")
display(query_df('''
SELECT 
    p.product_name,
    p.category,
    COUNT(oi.order_item_id) AS times_ordered,
    COALESCE(SUM(oi.quantity), 0) AS total_quantity
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_id, p.product_name, p.category
ORDER BY times_ordered DESC;
'''))

# Finding products that have NEVER been ordered
print("\nProducts never ordered (using LEFT JOIN + NULL check):")
display(query_df('''
SELECT p.product_name, p.category
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
WHERE oi.order_item_id IS NULL;
'''))
# Note: If the result is empty, all products have been ordered at least once!

### ✏️ Exercise 2: Working with Joins

Try these queries:
1. Calculate the total revenue per delivered order (join orders → order_items → products, filter by status)
2. List all customers and how many orders they've placed (including customers with 0 orders)
3. Find the most popular product category by total quantity sold

In [ ]:
# Exercise 2 Solutions

# 1) Revenue per delivered order
print("1. Revenue per delivered order:")
display(query_df('''
SELECT
  o.order_id,
  c.name AS customer_name,
  SUM(oi.quantity * p.unit_price) AS order_revenue
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE o.status = 'delivered'
GROUP BY o.order_id, c.name
ORDER BY order_revenue DESC;
'''))

# 2) All customers with order counts (including 0)
print("\n2. Customers with order counts (LEFT JOIN):")
display(query_df('''
SELECT 
    c.name AS customer_name,
    c.city,
    COUNT(o.order_id) AS order_count
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.name, c.city
ORDER BY order_count DESC;
'''))

# 3) Most popular category by quantity
print("\n3. Most popular category by quantity:")
display(query_df('''
SELECT 
    p.category,
    SUM(oi.quantity) AS total_quantity
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.category
ORDER BY total_quantity DESC;
'''))

## 9.7 Subqueries (queries inside queries)

A **subquery** is a query used inside another query.
It’s helpful when you want to compute an intermediate result first (like totals per customer), then filter or sort it.

Example question: “Which customers spent more than 500 on delivered orders?”

In [ ]:
# Subquery example: customers with delivered spending > 500
q = '''
SELECT customer_name, total_spent
FROM (
  SELECT
    c.name AS customer_name,
    SUM(oi.quantity * p.unit_price) AS total_spent
  FROM orders o
  JOIN customers c ON o.customer_id = c.customer_id
  JOIN order_items oi ON o.order_id = oi.order_id
  JOIN products p ON oi.product_id = p.product_id
  WHERE o.status = 'delivered'
  GROUP BY c.name
) t
WHERE total_spent > 500
ORDER BY total_spent DESC;
'''

display(query_df(q))

# Another pattern: IN (subquery)
q = '''
SELECT order_id, customer_id, order_date, status
FROM orders
WHERE customer_id IN (
  SELECT customer_id
  FROM customers
  WHERE city = 'Bengaluru'
)
ORDER BY order_date;
'''

display(query_df(q))

## 9.8 Views (Reusable Queries)

A **view** is a saved SQL query that acts like a virtual table.

### Why Use Views?

| Benefit | Explanation |
|---------|-------------|
| **Reusability** | Write complex joins once, reuse many times |
| **Simplicity** | Complex logic hidden; analysts query simple view |
| **Consistency** | Standard business logic (e.g., "delivered revenue") |
| **Security** | Can restrict access to underlying tables |

### How Views Work

```sql
-- Create a view
CREATE VIEW v_sales_summary AS
SELECT ... complex query ...;

-- Use it like a table
SELECT * FROM v_sales_summary WHERE region = 'North';
```

> **💡 Note:** Views store the **query definition**, not the data. When you query a view, the database runs the underlying query.

> **⚠️ Common Misconception:** Views don't automatically improve performance. They're primarily for organization and reusability.

In [ ]:
# Create a view of delivered line items (a great base for analytics)
conn.execute('DROP VIEW IF EXISTS v_delivered_line_items;')

conn.execute('''
CREATE VIEW v_delivered_line_items AS
SELECT
  o.order_id,
  o.order_date,
  c.customer_id,
  c.name AS customer_name,
  c.city,
  p.product_id,
  p.product_name,
  p.category,
  oi.quantity,
  p.unit_price,
  (oi.quantity * p.unit_price) AS line_total
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE o.status = 'delivered';
''')

# Query the view like a normal table
q = '''
SELECT *
FROM v_delivered_line_items
ORDER BY order_id, product_name;
'''

display(query_df(q))

### Exercise 4 (views)

Using `v_delivered_line_items`, answer:
1) Revenue by product category
2) Top 3 products by delivered revenue

Tip: This is where views shine—you can focus on business questions instead of repeating joins.

In [ ]:
# 1) Revenue by category
category_revenue = query_df('''
SELECT category, SUM(line_total) AS revenue
FROM v_delivered_line_items
GROUP BY category
ORDER BY revenue DESC;
''')

display(category_revenue)

# 2) Top 3 products by delivered revenue
top_products = query_df('''
SELECT product_name, SUM(line_total) AS revenue
FROM v_delivered_line_items
GROUP BY product_name
ORDER BY revenue DESC
LIMIT 3;
''')

display(top_products)

In [ ]:
# Visualizing SQL query results
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Chart 1: Revenue by category
ax1 = category_revenue.set_index('category')['revenue'].plot(
    kind='bar',
    ax=axes[0],
    color=['#2ecc71', '#3498db', '#e74c3c'],
    edgecolor='black'
)
ax1.set_title('Delivered Revenue by Category', fontsize=12, fontweight='bold')
ax1.set_xlabel('Category')
ax1.set_ylabel('Revenue (₹)')
ax1.tick_params(axis='x', rotation=45)

# Chart 2: Top products by revenue
ax2 = top_products.set_index('product_name')['revenue'].plot(
    kind='barh',
    ax=axes[1],
    color='#9b59b6',
    edgecolor='black'
)
ax2.set_title('Top 3 Products by Revenue', fontsize=12, fontweight='bold')
ax2.set_xlabel('Revenue (₹)')
ax2.set_ylabel('Product')

plt.tight_layout()
plt.show()

print("\n💡 Tip: pandas + SQL + matplotlib is a powerful combination for data analysis!")

## 9.9 Indexes and Query Performance

An **index** is a data structure that speeds up data retrieval, like the index in a textbook.

### How Indexes Work

Without index: Database scans **every row** (slow for large tables)
With index: Database jumps directly to matching rows (fast!)

### When to Create Indexes

| Good Candidates | Why |
|-----------------|-----|
| Columns in `WHERE` | Filter faster |
| Columns in `JOIN ON` | Match rows faster |
| Columns in `ORDER BY` | Pre-sorted access |
| Primary keys | Automatically indexed |

### Trade-offs

| Pros | Cons |
|------|------|
| ⚡ Faster reads (SELECT) | 🐢 Slower writes (INSERT/UPDATE/DELETE) |
| Better query performance | Uses more disk space |
| Essential for large tables | Index maintenance overhead |

> **💡 Tip for Beginners:** Focus on understanding indexes conceptually. For small datasets, you won't notice the difference. Indexes matter when you have thousands or millions of rows.

### Query Optimization Checklist

1. ✅ Select only columns you need (avoid `SELECT *` in production)
2. ✅ Filter early with `WHERE`
3. ✅ Ensure join keys are indexed
4. ✅ Use `LIMIT` when testing on large datasets
5. ✅ Use `EXPLAIN QUERY PLAN` to understand query execution

In [ ]:
# Create a couple of indexes (helps joins/filters when tables get large)
conn.execute('CREATE INDEX IF NOT EXISTS idx_orders_customer_id ON orders(customer_id);')
conn.execute('CREATE INDEX IF NOT EXISTS idx_order_items_order_id ON order_items(order_id);')
conn.execute('CREATE INDEX IF NOT EXISTS idx_order_items_product_id ON order_items(product_id);')
conn.commit()

print('Indexes created.')

# Use EXPLAIN QUERY PLAN to see (roughly) how SQLite will execute a query
plan = query_df('''
EXPLAIN QUERY PLAN
SELECT o.order_id, c.name
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE c.city = ?
ORDER BY o.order_date;
''', params=('Pune',))

display(plan)

## 9.10 Connecting Python to Databases

Python can connect to virtually any database. The concepts are the same across databases.

### Python Database Workflow

```
┌─────────────────┐     ┌──────────────┐     ┌──────────────┐
│  Python Code    │────▶│  Connection  │────▶│   Database   │
│ (sqlite3/pandas)│◀────│   Object     │◀────│   (SQLite)   │
└─────────────────┘     └──────────────┘     └──────────────┘
```

### Reading vs Writing

| Operation | Method | Commit Required? |
|-----------|--------|-----------------|
| **Read** (SELECT) | `pd.read_sql_query()` | No |
| **Write** (INSERT/UPDATE/DELETE) | `conn.execute()` | Yes! |

### Parameterized Queries (CRITICAL!)

**Never build SQL strings with concatenation!**

```python
# ❌ DANGEROUS - SQL Injection risk!
city = "Pune'; DROP TABLE customers; --"
query = f"SELECT * FROM customers WHERE city = '{city}'"

# ✅ SAFE - Use parameters
query = "SELECT * FROM customers WHERE city = ?"
conn.execute(query, (city,))
```

> **⚠️ SQL Injection** is a major security vulnerability. Always use parameterized queries to protect your applications.

### Database Libraries in Python

| Library | Use Case |
|---------|----------|
| `sqlite3` | SQLite (built-in, no install) |
| `psycopg2` | PostgreSQL |
| `mysql-connector` | MySQL |
| `pyodbc` | SQL Server, many others |
| `SQLAlchemy` | Universal ORM (works with all databases) |

In [ ]:
# Example: Parameterized SELECT (safe approach)
city = 'Delhi'
q = '''
SELECT customer_id, name, city
FROM customers
WHERE city = ?
ORDER BY name;
'''
print(f"Customers from {city}:")
display(query_df(q, params=(city,)))

# Example: INSERT new data
print("\nInserting a new customer...")
conn.execute(
    'INSERT INTO customers VALUES (?, ?, ?, ?);',
    (5, 'Eva', 'Chennai', '2025-04-15')
)
conn.commit()  # Don't forget to commit!

# Verify the insert
print("Customer list after insert:")
display(query_df('SELECT * FROM customers ORDER BY customer_id;'))

# Example: UPDATE existing data
print("\nUpdating order 105 status to 'delivered'...")
conn.execute(
    'UPDATE orders SET status = ? WHERE order_id = ?;',
    ('delivered', 105)
)
conn.commit()

# Verify the update
print("Order 105 after update:")
display(query_df('SELECT * FROM orders WHERE order_id = 105;'))

# Example: DELETE (be careful!)
print("\nDeleting customer Eva...")
conn.execute('DELETE FROM customers WHERE customer_id = ?;', (5,))
conn.commit()
print("Customer deleted successfully.")

### Transactions: All or Nothing

A **transaction** groups multiple operations that should either all succeed or all fail.

```python
try:
    conn.execute('INSERT INTO orders ...')
    conn.execute('INSERT INTO order_items ...')
    conn.commit()  # All succeeded - save changes
except Exception as e:
    conn.rollback()  # Something failed - undo all changes
```

> **💡 Tip:** SQLite auto-commits after each statement by default. Use explicit transactions for related operations.

## 🎯 Mini-Project: Building a Sales Report

Now let's put everything together! You'll build a comprehensive sales report using the skills from this chapter.

### Your Tasks

Using the `v_delivered_line_items` view (or writing your own joins), create:

1. **Total delivered revenue** - One number for all delivered orders
2. **Revenue by city** - Which city generates the most revenue?
3. **Revenue by month** - Format as `YYYY-MM`
4. **Top 3 customers** - Who spent the most on delivered orders?

### Bonus Challenges
5. **Average order value** - What's the average revenue per delivered order?
6. **Product performance** - Revenue and quantity for each product
7. **Create a visualization** - Plot the results using matplotlib

> **💡 Tip:** Start from `v_delivered_line_items` to avoid rewriting joins. Try it yourself first before looking at the solutions below!

In [ ]:
# Mini-Project Solutions

# 1) Total delivered revenue
print("=" * 50)
print("1. TOTAL DELIVERED REVENUE")
print("=" * 50)
total_revenue = query_df('''
SELECT SUM(line_total) AS total_delivered_revenue
FROM v_delivered_line_items;
''')
display(total_revenue)

# 2) Revenue by city
print("\n" + "=" * 50)
print("2. REVENUE BY CITY")
print("=" * 50)
revenue_by_city = query_df('''
SELECT city, SUM(line_total) AS revenue
FROM v_delivered_line_items
GROUP BY city
ORDER BY revenue DESC;
''')
display(revenue_by_city)

# 3) Revenue by month
print("\n" + "=" * 50)
print("3. REVENUE BY MONTH")
print("=" * 50)
revenue_by_month = query_df('''
SELECT SUBSTR(order_date, 1, 7) AS month, SUM(line_total) AS revenue
FROM v_delivered_line_items
GROUP BY SUBSTR(order_date, 1, 7)
ORDER BY month;
''')
display(revenue_by_month)

# 4) Top 3 customers
print("\n" + "=" * 50)
print("4. TOP 3 CUSTOMERS BY REVENUE")
print("=" * 50)
top_customers = query_df('''
SELECT customer_name, SUM(line_total) AS revenue
FROM v_delivered_line_items
GROUP BY customer_name
ORDER BY revenue DESC
LIMIT 3;
''')
display(top_customers)

In [ ]:
# Bonus Solutions

# 5) Average order value
print("=" * 50)
print("5. AVERAGE ORDER VALUE")
print("=" * 50)
avg_order_value = query_df('''
SELECT 
    COUNT(DISTINCT order_id) AS num_orders,
    SUM(line_total) AS total_revenue,
    ROUND(SUM(line_total) * 1.0 / COUNT(DISTINCT order_id), 2) AS avg_order_value
FROM v_delivered_line_items;
''')
display(avg_order_value)

# 6) Product performance
print("\n" + "=" * 50)
print("6. PRODUCT PERFORMANCE")
print("=" * 50)
product_performance = query_df('''
SELECT 
    product_name,
    SUM(quantity) AS total_quantity,
    SUM(line_total) AS total_revenue
FROM v_delivered_line_items
GROUP BY product_name
ORDER BY total_revenue DESC;
''')
display(product_performance)

In [ ]:
# 7) Mini-Project Visualization
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Revenue by City
ax1 = revenue_by_city.plot(
    kind='pie', 
    y='revenue', 
    labels=revenue_by_city['city'],
    ax=axes[0, 0],
    autopct='%1.1f%%',
    legend=False
)
ax1.set_title('Revenue Distribution by City', fontweight='bold')
ax1.set_ylabel('')

# Top Customers
ax2 = top_customers.set_index('customer_name')['revenue'].plot(
    kind='barh',
    ax=axes[0, 1],
    color='#3498db',
    edgecolor='black'
)
ax2.set_title('Top 3 Customers by Revenue', fontweight='bold')
ax2.set_xlabel('Revenue (₹)')

# Revenue by Month
ax3 = revenue_by_month.plot(
    x='month',
    y='revenue',
    kind='bar',
    ax=axes[1, 0],
    color='#2ecc71',
    edgecolor='black',
    legend=False
)
ax3.set_title('Monthly Revenue Trend', fontweight='bold')
ax3.set_xlabel('Month')
ax3.set_ylabel('Revenue (₹)')
ax3.tick_params(axis='x', rotation=0)

# Product Performance
ax4 = product_performance.set_index('product_name')['total_revenue'].plot(
    kind='bar',
    ax=axes[1, 1],
    color='#9b59b6',
    edgecolor='black'
)
ax4.set_title('Revenue by Product', fontweight='bold')
ax4.set_xlabel('Product')
ax4.set_ylabel('Revenue (₹)')
ax4.tick_params(axis='x', rotation=45)

plt.suptitle('Sales Report Dashboard', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n✅ Mini-project complete! You've built a comprehensive sales report using SQL and Python.")

## Summary / Key Takeaways

### Core Concepts
- ✅ **Databases** store data in **tables** with rows (records) and columns (fields)
- ✅ **Primary keys** uniquely identify rows; **foreign keys** link tables
- ✅ **Normalization** reduces data duplication by storing facts once

### SQL Essentials
- ✅ `SELECT ... FROM ... WHERE ... ORDER BY` - the basic query pattern
- ✅ `GROUP BY` + aggregate functions (`COUNT`, `SUM`, `AVG`) for summaries
- ✅ `WHERE` filters rows **before** grouping; `HAVING` filters **after**
- ✅ `JOIN` combines tables; `INNER JOIN` for matches only, `LEFT JOIN` for all left + matches
- ✅ Subqueries help break complex problems into steps
- ✅ Views save reusable query logic

### Python + Databases
- ✅ Use `sqlite3` for SQLite connections
- ✅ Use `pandas.read_sql_query()` for reading data
- ✅ **Always use parameterized queries** (prevent SQL injection!)
- ✅ Remember to `commit()` after INSERT/UPDATE/DELETE
- ✅ Indexes improve query performance for large datasets

### Common Mistakes to Avoid
| Mistake | Solution |
|---------|----------|
| Using `WHERE` with aggregates | Use `HAVING` instead |
| Forgetting `commit()` after writes | Always commit changes |
| String concatenation in SQL | Use parameterized queries |
| `SELECT *` in production | Select specific columns |
| Missing join conditions | Always specify `ON` clause |

## Additional Resources

### Official Documentation
- [SQLite Documentation](https://www.sqlite.org/docs.html) - Official SQLite reference
- [Python sqlite3 Module](https://docs.python.org/3/library/sqlite3.html) - Python's built-in SQLite library
- [pandas read_sql_query](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html) - pandas SQL integration

### Tutorials and Learning
- [W3Schools SQL Tutorial](https://www.w3schools.com/sql/) - Interactive SQL basics
- [SQLBolt](https://sqlbolt.com/) - Learn SQL with interactive exercises
- [Mode SQL Tutorial](https://mode.com/sql-tutorial/) - Comprehensive SQL for analytics

### Advanced Topics
- [SQLite Query Planner (EXPLAIN)](https://www.sqlite.org/eqp.html) - Understanding query execution
- [SQLAlchemy Documentation](https://docs.sqlalchemy.org/) - Python SQL toolkit and ORM

### Practice Platforms
- [LeetCode SQL Problems](https://leetcode.com/problemset/database/) - SQL interview practice
- [HackerRank SQL](https://www.hackerrank.com/domains/sql) - SQL challenges by difficulty

In [ ]:
# Clean up: close the database connection
conn.close()
print("Database connection closed.")
print("\n🎉 Congratulations! You've completed Chapter 9: Database Systems and SQL for Data Analysis.")